# Loading_Data
1. Loding single trajectory for single user.
2. Loading all trajectories for single user.
3. Loading all user.
4. Keeping sample=10 for analysis.
5. Calculating difference in time stamp for consective location for each user.

In [1]:
import pandas as pd

def load_single_trajectory(file_path):
    df = pd.read_csv(
        file_path,
        skiprows=6,
        header=None
    )

    df.columns = [
        'lat',
        'lon',
        'unused',
        'altitude',
        'days',
        'date',
        'time'
    ]

    df['timestamp'] = pd.to_datetime(
        df['date'] + ' ' + df['time'],
        errors='coerce'
    )

    df = df.dropna(subset=['timestamp'])

    return df[['lat', 'lon', 'altitude', 'timestamp']]


In [2]:
from pathlib import Path

def load_user_trajectories(user_path):
    all_trips = []

    traj_path = Path(user_path) / "Trajectory"
    for file in traj_path.glob("*.plt"):
        df = load_single_trajectory(file)
        df['source_file'] = file.name
        all_trips.append(df)

    return pd.concat(all_trips, ignore_index=True)


In [6]:
user_000 = load_user_trajectories("D:\\projects\\gps_project\\Data\\raw\\000")
print(user_000.head())


         lat         lon  altitude           timestamp         source_file
0  39.984702  116.318417       492 2008-10-23 02:53:04  20081023025304.plt
1  39.984683  116.318450       492 2008-10-23 02:53:10  20081023025304.plt
2  39.984686  116.318417       492 2008-10-23 02:53:15  20081023025304.plt
3  39.984688  116.318385       492 2008-10-23 02:53:20  20081023025304.plt
4  39.984655  116.318263       492 2008-10-23 02:53:25  20081023025304.plt


In [7]:
def load_all_users(base_path, max_users=None):
    users = []
    base = Path(base_path)

    user_dirs = sorted(base.iterdir())
    if max_users:
        user_dirs = user_dirs[:max_users]

    for user_dir in user_dirs:
        df = load_user_trajectories(user_dir)
        df['user_id'] = user_dir.name
        users.append(df)

    return pd.concat(users, ignore_index=True)


In [9]:
gps_raw = load_all_users("D:\\projects\\gps_project\\Data\\raw", max_users=50)

In [10]:
gps_raw = gps_raw.reset_index(drop=True)

In [11]:
gps_raw.info()
gps_raw.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10776492 entries, 0 to 10776491
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   lat          float64       
 1   lon          float64       
 2   altitude     float64       
 3   timestamp    datetime64[ns]
 4   source_file  object        
 5   user_id      object        
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 493.3+ MB


,lat,lon,altitude,timestamp
count,1.077649e+07,1.077649e+07,1.077649e+07,10776492
mean,3.920098e+01,1.159208e+02,4.491132e+02,2009-03-04 01:23:27.889501440
min,1.816888e+01,7.598031e+01,-3.264760e+04,2007-04-29 08:34:32
25%,3.989377e+01,1.163077e+02,1.140000e+02,2008-12-19 20:51:33.750000128
50%,3.997195e+01,1.163330e+02,1.630000e+02,2009-02-28 09:36:37
75%,3.999926e+01,1.164029e+02,2.270000e+02,2009-04-30 07:08:48.249999872
max,4.001667e+02,1.428796e+02,1.075033e+05,2011-12-27 05:18:16
std,2.981448e+00,3.709628e+00,1.994736e+03,NaN


In [12]:
gps_raw = gps_raw.sort_values(
    by=['user_id', 'timestamp']
).reset_index(drop=True)


In [13]:
gps_raw.head()

,lat,lon,altitude,timestamp,source_file,user_id
0,39.984702,116.318417,492.0,2008-10-23 02:53:04,20081023025304.plt,000
1,39.984683,116.318450,492.0,2008-10-23 02:53:10,20081023025304.plt,000
2,39.984686,116.318417,492.0,2008-10-23 02:53:15,20081023025304.plt,000
3,39.984688,116.318385,492.0,2008-10-23 02:53:20,20081023025304.plt,000
4,39.984655,116.318263,492.0,2008-10-23 02:53:25,20081023025304.plt,000


In [14]:
gps_raw['delta_time_s'] = (
    gps_raw
    .groupby('user_id')['timestamp']
    .diff()
    .dt.total_seconds().reset_index(drop=True)
)


In [ ]:
gps_raw.head()

In [15]:
gps_raw['delta_time_s'] = gps_raw['delta_time_s'].fillna(0)
# first row has no previous time stamps for each user to compute distance hence filling with 0

In [16]:
gps_raw['large_gap'] = gps_raw['delta_time_s'] > 1800
#large gap of more then 30 min

In [17]:
gps_raw['large_gap'].mean()

np.float64(0.001075581924062116)

In [18]:
gps_raw['delta_time_s'].describe()


count    1.077649e+07
mean     5.032493e+01
std      9.490226e+03
min      0.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      5.000000e+00
max      1.795909e+07
Name: delta_time_s, dtype: float64

## Distance Computation

Distance between consecutive GPS points is computed using a custom implementation
of the Haversine formula. This avoids reliance on external geospatial libraries
and ensures portability across environments.


In [19]:
import numpy as np

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters

    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)

    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)

    a = (
        np.sin(dphi / 2)**2 +
        np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    )

    return 2 * R * np.arcsin(np.sqrt(a))



In [22]:
def compute_distance(df):
    df = df.reset_index(drop=True)

    lat1 = df["lat"].values[:-1]
    lon1 = df["lon"].values[:-1]
    lat2 = df["lat"].values[1:]
    lon2 = df["lon"].values[1:]

    distances = np.zeros(len(df))
    distances[1:] = haversine_distance(lat1, lon1, lat2, lon2)

    df["delta_dist_m"] = distances
    return df



In [23]:
gps_raw = (
    gps_raw
    .groupby('user_id', group_keys=False)
    .apply(compute_distance).reset_index(drop=True)
)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_3772\3043923854.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_distance).reset_index(drop=True)


In [24]:
gps_raw.head()

,lat,lon,altitude,timestamp,source_file,user_id,delta_time_s,large_gap,delta_dist_m
0,39.984702,116.318417,492.0,2008-10-23 02:53:04,20081023025304.plt,000,0.0,False,0.000000
1,39.984683,116.318450,492.0,2008-10-23 02:53:10,20081023025304.plt,000,6.0,False,3.516886
2,39.984686,116.318417,492.0,2008-10-23 02:53:15,20081023025304.plt,000,5.0,False,2.831299
3,39.984688,116.318385,492.0,2008-10-23 02:53:20,20081023025304.plt,000,5.0,False,2.735434
4,39.984655,116.318263,492.0,2008-10-23 02:53:25,20081023025304.plt,000,5.0,False,11.023008


## Speed Estimation

Instantaneous speed is calculated as distance divided by time difference between
consecutive GPS points. Observations with zero time difference are assigned zero
speed to avoid division errors.

Unrealistic speeds (>60 m/s) are treated as noise and excluded from analysis.


In [25]:
gps_raw['speed_mps'] = (
    gps_raw['delta_dist_m'] / gps_raw['delta_time_s']
)

gps_raw.loc[gps_raw['delta_time_s'] == 0, 'speed_mps'] = 0


In [26]:
gps_raw.loc[gps_raw['speed_mps'] > 60, 'speed_mps'] = None


## Movement Detection

A point is classified as movement if instantaneous speed exceeds 0.5 m/s.
This threshold filters GPS jitter while preserving slow human motion such as walking.


In [27]:
gps_raw['is_moving'] = gps_raw['speed_mps'] > 0.5


## Data Quality Checks

Summary statistics and movement proportions were inspected to validate
distance and speed calculations and identify anomalies.


In [28]:
gps_raw[['delta_dist_m', 'speed_mps']].describe()


,delta_dist_m,speed_mps
count,1.077649e+07,1.076102e+07
mean,2.985958e+01,8.107478e+00
std,3.422937e+03,9.054643e+00
min,0.000000e+00,0.000000e+00
25%,3.555245e+00,1.074240e+00
50%,1.005965e+01,4.305592e+00
75%,1.713321e+01,1.291241e+01
max,3.017062e+06,5.999725e+01


In [29]:
gps_raw['is_moving'].value_counts(normalize=True)


is_moving
True     0.831003
False    0.168997
Name: proportion, dtype: float64

## Trip Boundary Logic

Trip boundaries are identified using a rule-based approach that combines
temporal gaps and motion characteristics. A new trip is initiated when
either a large time gap (>30 minutes) is observed or when prolonged
stationarity (>10 minutes with speed <0.5 m/s) occurs.


In [30]:
gps_raw['trip_break'] = (
    (gps_raw['delta_time_s'] > 1800) |
    (
        (gps_raw['speed_mps'] < 0.5) &
        (gps_raw['delta_time_s'] > 600)
    )
)


## Trip Identifier Construction

Trip identifiers are generated per user using a cumulative sum over detected
trip boundaries. This ensures sequential, interpretable trip IDs without
reliance on file structure or external metadata.



In [31]:
gps_raw['trip_id'] = (
    gps_raw
    .groupby('user_id')['trip_break']
    .cumsum()
)


## Trip Validation

Very short or noisy trips are filtered out using minimum thresholds on
number of points and total distance. This removes GPS artifacts while
preserving meaningful mobility behavior.


In [32]:
trip_stats = (
    gps_raw
    .groupby(['user_id', 'trip_id'])
    .agg(
        points=('timestamp', 'count'),
        total_dist=('delta_dist_m', 'sum'),
        duration=('delta_time_s', 'sum')
    )
    .reset_index()
)

valid_trips = trip_stats[
    (trip_stats['points'] >= 10) &
    (trip_stats['total_dist'] >= 100)
]


In [38]:
gps_raw.to_csv("D:\\projects\\gps_project\\Data\\staging\\gps_enriched.csv", index=False)


In [39]:
gps_raw

,lat,lon,altitude,timestamp,source_file,user_id,delta_time_s,large_gap,delta_dist_m,speed_mps,is_moving,trip_break,trip_id
0,39.984702,116.318417,492.000000,2008-10-23 02:53:04,20081023025304.plt,000,0.0,False,0.000000,0.000000,False,False,0
1,39.984683,116.318450,492.000000,2008-10-23 02:53:10,20081023025304.plt,000,6.0,False,3.516886,0.586148,True,False,0
2,39.984686,116.318417,492.000000,2008-10-23 02:53:15,20081023025304.plt,000,5.0,False,2.831299,0.566260,True,False,0
3,39.984688,116.318385,492.000000,2008-10-23 02:53:20,20081023025304.plt,000,5.0,False,2.735434,0.547087,True,False,0
4,39.984655,116.318263,492.000000,2008-10-23 02:53:25,20081023025304.plt,000,5.0,False,11.023008,2.204602,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10776487,40.967805,117.930953,1055.282431,2009-09-02 17:20:15,20090902130325.plt,049,5.0,False,2.644217,0.528843,True,False,2
10776488,40.967774,117.930981,1054.833780,2009-09-02 17:20:20,20090902130325.plt,049,5.0,False,4.172392,0.834478,True,False,2
10776489,40.967689,117.930989,1054.067536,2009-09-02 17:20:25,20090902130325.plt,049,5.0,False,9.475406,1.895081,True,False,2
10776490,40.967873,117.930966,1052.288947,2009-09-02 17:24:25,20090902130325.plt,049,240.0,False,20.550798,0.085628,False,False,2
